In [1]:
!pip install datasets -q
!pip install tokenizers -q
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 67.0 MB/s eta 0:00:00


In [2]:
import os
import datasets
from tqdm.notebook import tqdm

let's use the OSCAR dataset and get the hebrew training portion of it.

since it's huge i'll use the streaming mode and only get 200000 docs


In [3]:
dataset = datasets.load_dataset(
    'oscar',
    'unshuffled_deduplicated_he',
    split='train',
    streaming=True)

sampled_ds = []
for i, doc in tqdm(enumerate(dataset)):
  if i > 200_000:
    break
  sampled_ds.append(doc)

0it [00:00, ?it/s]

let's take a look at the data

In [4]:
sampled_ds[-10:]

[{'id': 199991,
  'text': 'בלילה האחרון החלו הקרנות הבכורה בארץ ובעולם של הסרט החדש "Star Wars The Force Awakens" ולכבוד הסרט גוגל בנו משחק קצרצר של מלחמת הכוכבים הפועל בתוך דפדפן הכרום.\nעליכם יהיה להכנס לקישור עם הקוד המיוחד שיוצג לכם באתר במחשב מהפלאפון ולאחר ביצוע כיול קצרצר ופשוט תוכלו להדוף יריות ולבסוף גם להלחם בStorm Trooper.\nלצערי המשקח מאוד מאוד קצר ומכיוון שמדובר במשחק דפדפן באיכות לא רעה ושליטה שניעשת ממכשיר אחר דרך הרשת יש מדי פעם קצת האטה.\nתגיות: גוגל, דפדפן, כרום, מלחמת הכוכבים, משחק, סרט, Chrome, Lightsaber, Star Wars, Storm Trooper, The Force Awakens'},
 {'id': 199992,
  'text': 'התרגלנו לחשוב שיש לנו סוג של בעלות על האדם האהוב עלינו. לפעמים מספיק שנראה את הבן זוג שלנו מסתכל על אישה אחרת, והקנאה מתעוררת. האמירה הבאה אולי תפתיע אתכם: קנאה אינה שייכת לאהבה! אם תאמצו זאת כמוטו, חיי האהבה שלכם ישתנו לחלוטין\nלרפואה המזרחית גישה מעניינת ואפקטיבית מאוד, לתופעות גיל המעבר. המחלקה לרפואה סינית ויפנית במרפאות אלטרנטיבה מציעה טיפול ייחודי משולב לתופעות המטרידות של גיל המעבר: ג

In [5]:
CORPUS_DIR = './oscar_heb'

if not os.path.exists(CORPUS_DIR):
  os.mkdir(CORPUS_DIR)

text_data = []
file_count = 0

for sample in tqdm(sampled_ds):
    
    # Replace all '\n' (new line) symbols, with the '\s' (white space) 
    # character, making each document a single line.
    sample = sample['text'].replace('\n', '\s')
    
    text_data.append(sample)
    if len(text_data) == 5_000:
        
        # For every 5,000 documents, save them to a file names "text_i" for i 
        # in 0-39. The documents in the file are seperated with a new line.
        # after each save, reinitialize text_data list and file_count 
        with open(f'{CORPUS_DIR}/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

# Write the remaining document to the last file 
with open(f'{CORPUS_DIR}/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))

  0%|          | 0/200001 [00:00<?, ?it/s]

In [6]:
from pathlib import Path
paths = [str(x) for x in Path(CORPUS_DIR).glob('**/*.txt')]
paths[:5]

['oscar_heb/text_12.txt',
 'oscar_heb/text_14.txt',
 'oscar_heb/text_16.txt',
 'oscar_heb/text_13.txt',
 'oscar_heb/text_36.txt']

let's init the standart tokenizer for BERT which is called WordPiece

In [7]:
from tokenizers import BertWordPieceTokenizer

# initialize
tokzr_trainer = BertWordPieceTokenizer(clean_text=True)

# and train
tokzr_trainer.train(files=paths, vocab_size=3_000, min_frequency=2,
                limit_alphabet=1_000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])

and save it

In [8]:
MODEL_DIR = './wp-heb'

if not os.path.exists(MODEL_DIR):
  os.mkdir(MODEL_DIR)

tokzr_trainer.save_model(MODEL_DIR, 'wp-heb')

['./wp-heb/wp-heb-vocab.txt']

let's look at some tokenizing examples:

In [9]:
from transformers import BertTokenizer

tokzr = BertTokenizer.from_pretrained(f'{MODEL_DIR}/wp-heb-vocab.txt')

print(f'There are {tokzr.vocab_size} tokens in our tokenizer\'s vocabulary.\n===')
print(tokzr.tokenize('hello world'))
print(tokzr.tokenize('שלום עולם'))
print(tokzr.tokenize('אם אין אני לי מי לי'))

There are 3000 tokens in our tokenizer's vocabulary.
===
['h', '##el', '##l', '##o', 'w', '##or', '##l', '##d']
['שלו', '##ם', 'עו', '##לם']
['אם', 'אין', 'אני', 'לי', 'מי', 'לי']


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:1706: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


## domain specfic tokenizer
let's say we look at the sport domain and we want to test how suited is the WordPiece tokenizer for the task

In [10]:
' '.join(tokzr.tokenize('מכבי הפועל כדורגל כדורסל כדורעף טניס'))

'מכ ##בי הפ ##ועל כד ##ור ##גל כד ##ור ##ס ##ל כד ##ור ##ע ##ף ט ##ני ##ס'

doesn't look so good
we'll change the training set of the tokenizer to create a better tokenizer for the sport domain

In [11]:
SPORTS_WORDS = 'מכבי הפועל כדורגל כדורסל כדורעף טניס'.split()
SPORTS_CORPUS_DIR = './sports'
SPORTS_MODEL_DIR = './sp-heb'

# Load all the HEB data from OSCAR
dataset_1 = datasets.load_dataset(
    'oscar',
    'unshuffled_deduplicated_he',
    split='train',
    streaming=True)

sport_ds = []
doc_count = 0
for doc in tqdm(dataset_1):
  if len(sport_ds) > 200000:
    break
  # Take documents with at least 2 instances of one of the sport words.
  if any(doc['text'].count(sport_word) > 1 for sport_word in SPORTS_WORDS):
    sport_ds.append(doc)
print(f'There are {len(sport_ds)} soprt documents.')

# Save the documents
if not os.path.exists(SPORTS_CORPUS_DIR):
  os.mkdir(SPORTS_CORPUS_DIR)

text_data = []
file_count = 0
for sample in tqdm(sport_ds):
    # Replace all '\n' (new line) chars, with the '\s' (white space) 
    # character, making each document a single line.
    sample = sample['text'].replace('\n', '\s')
    # sample = sample.replace('\n', '\s')
    text_data.append(sample)
    if len(text_data) == 5_000:
        # For every 5,000 documents, save them to a file names "text_i".
        # The documents in the file are seperated with a new line.
        with open(f'{SPORTS_CORPUS_DIR}/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1
# Write the remaining document to the last file .
with open(f'{SPORTS_CORPUS_DIR}/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
    fp.write('\n'.join(text_data))
sport_paths = [str(x) for x in Path(SPORTS_CORPUS_DIR).glob('**/*.txt')]

# Initialize, train and save the model.
sports_tokzr_trainer = BertWordPieceTokenizer(clean_text=True)
sports_tokzr_trainer.train(files=sport_paths, vocab_size=3_000, min_frequency=2,
                limit_alphabet=1_000, wordpieces_prefix='##',
                special_tokens=[
                    '[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]'])
if not os.path.exists(SPORTS_MODEL_DIR):
  os.mkdir(SPORTS_MODEL_DIR)
sports_tokzr_trainer.save_model(SPORTS_MODEL_DIR, 'wp-heb')

sports_tokzr = BertTokenizer.from_pretrained(f'{SPORTS_MODEL_DIR}/wp-heb-vocab.txt')

' '.join(sports_tokzr.tokenize('מכבי הפועל כדורגל כדורסל כדורעף טניס'))

0it [00:00, ?it/s]

There are 62555 soprt documents.


  0%|          | 0/62555 [00:00<?, ?it/s]

'מכבי הפועל כדור ##גל כדור ##סל כדור ##ע ##ף ט ##ניס'

a greedy inefficient algorithm for the tokenizing from the tokens vocab:

In [12]:
from typing import Callable, List

class GreedyTokenizerFromFile(Callable):
  
  def __init__(self, vocab_file: str) -> None:
    self.vocab = set()

    with open(vocab_file, 'r') as f:
      input_vocab = f.read().split("\n")
    # Add all tokens from the vocab file to our vocabulary. Our vocab is a set
    # so each word is unique.
    for token in input_vocab:
      self.vocab.add(token)
  
  def __call__(self, s: str) -> List:

    def get_longest_prefix(s):
      for i in range(len(s)):
        prefix = s[:len(s) - i]
        if any(token == prefix for token in self.vocab):
          return prefix
      return ""

    out = []
    i = 0
    input_len = len(s)
    if (input_len == 1):
      return [s]
    while i < input_len:
      longest_prefix = get_longest_prefix(s)
      out.append(longest_prefix)
      s = "##" + s[len(longest_prefix):]
      i += len(longest_prefix) - 2 if longest_prefix.startswith('##') else len(longest_prefix)
    return out
  

In [13]:
from tokenizers.pre_tokenizers import BertPreTokenizer

greedy_tok = GreedyTokenizerFromFile(f'{MODEL_DIR}/wp-heb-vocab.txt')

# Bert's PreTokenizer handles punctuation. It's called implicitly by tokzr
pretok = BertPreTokenizer()

# our "test set" is a random document from the dataset
for w, char_span in tqdm(pretok.pre_tokenize_str(sampled_ds[2023]['text'])):

  # ground truth - the automatically-learned tokenization
  trained_tok = tokzr.tokenize(w)

  # preparation - BertTokenizer actually has a preprocessing submodule called
  #               "basic_tokenizer" that lowercases English and such
  grd_tok = greedy_tok(tokzr.basic_tokenizer.tokenize(w)[0])
  
  # Test - these asserts should not fail #
  assert trained_tok == grd_tok, f'{w}, {trained_tok}, {grd_tok}'

  0%|          | 0/25 [00:00<?, ?it/s]